In [1]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
import pandas as pd
from tqdm import tqdm

In [ ]:
biz = []

all_business_links = []

total_businesses = 0

path = "C:/Users/HP/Downloads/chromedriver-win64/chromedriver.exe"

service = Service(path)
driver = webdriver.Chrome(service=service)

driver.set_page_load_timeout(120)

try: 
    url = 'https://ngxgroup.com/exchange/trade/equities/listed-companies/'
    driver.get(url) 

    while True: 
        time.sleep(5)

        tables = driver.find_elements(By.XPATH, '//tbody[@id="ngx_companies_listed_securities"]')

        for table in tables:
            rows = table.find_elements(By.TAG_NAME, "tr")

            for row in rows:
                cells = row.find_elements(By.TAG_NAME, "td")

                for cell in cells:
                    try:
                        business_links = [link.get_attribute('href') for link in cell.find_elements(By.TAG_NAME, 'a')]
                        all_business_links.extend(business_links)
                    except Exception:
                        print(f"Business, {cell.text} does not have a hyperlink")
        
        next_button = WebDriverWait(driver,10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@id='latestdiclosuresListed_next']"))
        )

        # if next_button.is_enabled()==True: 
        if "disabled" in next_button.get_attribute("class"):
            break  
        else:
            next_button.click()
            time.sleep(5)

    with tqdm(total=len(all_business_links), desc="Scraping Business Profiles") as pbar:
        for link in all_business_links:
            driver.get(link)

            time.sleep(10)

            business_profile = {}

            # page_source = driver.page_source

            sheets = driver.find_elements(By.XPATH, '//div[@class="content"]')
            for sheet in sheets:
                lines = sheet.find_elements(By.TAG_NAME, 'tr')

                for line in lines:
                    columns = line.find_elements(By.TAG_NAME, 'td')
                    
                    if len(columns) >= 1:
                        header = columns[0].text.strip() 

                        # If second column exists, get its value; otherwise, store `None`
                        value = columns[1].text.strip() if len(columns) > 1 else None

                        business_profile[header] = value

                        biz.append(business_profile)

            total_businesses +=1
            pbar.update(1)


finally:
    driver.quit()

In [40]:
biz

[{'Company Name:': 'CAVERTON OFFSHORE SUPPORT GRP PLC',
  'Ticker:': 'CAVERTON',
  'Market Classification:': 'Main Board',
  'Sector:': 'SERVICES',
  'Sub Sector:': 'Support and Logistics',
  'Nature of Business:': 'Marine & Aviation',
  'Company Address:': '1 Prince Kayode Akingbade Close, Victoria Island, Lagos',
  'Telephone:': '23412705656',
  'Fax:': '23414617645',
  'Email:': 'info@caverton-offshore.com',
  'Auditor:': '',
  'Registrar:': '',
  'Company Secretary:': '',
  'Date Listed:': 'Feb-06-2014',
  'Date of Incorporation:': 'Jun-02-2008',
  'Website:': 'www.caverton-offshore.com',
  'Board Of Directors:': '',
  '': None},
 {'Company Name:': 'CAVERTON OFFSHORE SUPPORT GRP PLC',
  'Ticker:': 'CAVERTON',
  'Market Classification:': 'Main Board',
  'Sector:': 'SERVICES',
  'Sub Sector:': 'Support and Logistics',
  'Nature of Business:': 'Marine & Aviation',
  'Company Address:': '1 Prince Kayode Akingbade Close, Victoria Island, Lagos',
  'Telephone:': '23412705656',
  'Fax:': 